# Clase 7 — Fundamentos IA: Estadística aplicada con Python

Este notebook está diseñado para usar en clase: incluye **muchos ejemplos**, explicaciones en Markdown y código listo para ejecutar. 
Contenidos:

- Estadística descriptiva (media, mediana, moda, varianza, desviación estándar, rango)
- Medidas de posición (cuartiles, percentiles)
- Distribuciones (normal, sesgada, bimodal, uniforme)
- Correlaciones y visualizaciones
- Correlación vs causalidad
- Detección e interpretación de outliers
- Caso práctico: E-commerce (ventas vs publicidad)

Ejecuta las celdas en orden. Todas las gráficas usan `matplotlib` (sin especificar colores) para garantizar consistencia didáctica.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

# Ajustes visuales mínimos
plt.rcParams['figure.figsize'] = (8,4)
plt.rcParams['font.size'] = 12

print('librerías cargadas')

## Dataset 1 — Salarios (ejemplo realista y pequeño)
Generaremos un dataset con variables típicas: `salario_kusd` (miles USD), `edad`, `experiencia_anios`, `nivel_educativo` (categoría), `departamento` (categoría).

In [ ]:
# Generar dataset de salarios
np.random.seed(42)
N = 200
edad = np.random.normal(35, 8, N).astype(int)
experiencia = np.clip((edad - 22) + np.random.normal(0, 3, N), 0, None).round(1)

# salario base según experiencia y educación
nivel_edu = np.random.choice(['Bachiller', 'Pregrado', 'Posgrado'], size=N, p=[0.15, 0.65, 0.20])
salario_kusd = 20 + experiencia * 1.3 + (np.where(nivel_edu=='Pregrado', 5, np.where(nivel_edu=='Posgrado', 12, 0)))
# añadir ruido y algunos outliers
salario_kusd = salario_kusd + np.random.normal(0, 5, N)
# forzar algunos outliers altos y bajos
salario_kusd[5] = 120  # outlier alto
salario_kusd[12] = 5   # outlier bajo

df_sal = pd.DataFrame({
    'salario_kusd': np.round(salario_kusd,2),
    'edad': edad,
    'experiencia_anios': experiencia,
    'nivel_educativo': nivel_edu,
    'departamento': np.random.choice(['Ventas','TI','Operaciones','Marketing'], size=N)
})

df_sal.head()

### Estadística descriptiva — ejemplos
Calcularemos media, mediana, moda, rango, varianza y desviación estándar. Interpretaremos diferencias entre media y mediana.

In [ ]:
# Estadísticos básicos
stats_sal = df_sal['salario_kusd'].describe()
media = df_sal['salario_kusd'].mean()
mediana = df_sal['salario_kusd'].median()
moda = df_sal['salario_kusd'].mode().iloc[0]
varianza = df_sal['salario_kusd'].var()
desv_std = df_sal['salario_kusd'].std()
rango = df_sal['salario_kusd'].max() - df_sal['salario_kusd'].min()

stats_sal, media, mediana, moda, varianza, desv_std, rango

### Visualizaciones: Histograma y boxplot
Mostraremos la forma de la distribución y detectaremos outliers con boxplot.

In [ ]:
# Histograma
plt.figure()
plt.hist(df_sal['salario_kusd'], bins=20)
plt.title('Histograma de salarios (miles USD)')
plt.xlabel('Salario (kUSD)')
plt.ylabel('Frecuencia')
plt.show()

# Boxplot
plt.figure()
plt.boxplot(df_sal['salario_kusd'], vert=False)
plt.title('Boxplot de salarios')
plt.xlabel('Salario (kUSD)')
plt.show()

**Interpretación — Media vs Mediana**

- Cuando existe un outlier (por ejemplo un salario extremadamente alto), la **media** se desplaza hacia ese valor mientras que la **mediana** se mantiene más robusta.
- Siempre reportar ambas métricas si la distribución no es simétrica.

In [ ]:
# Mostrar media y mediana en el dataset
print(f"Media: {media:.2f} kUSD")
print(f"Mediana: {mediana:.2f} kUSD")
print(f"Moda: {moda:.2f} kUSD")
print(f"Rango: {rango:.2f} kUSD")
print(f"Desviación estándar: {desv_std:.2f} kUSD")

### Medidas de posición: cuartiles y percentiles
Calcularemos cuartiles y percentiles y mostraremos ejemplos de interpretación.

In [ ]:
q1 = df_sal['salario_kusd'].quantile(0.25)
q2 = df_sal['salario_kusd'].quantile(0.5)
q3 = df_sal['salario_kusd'].quantile(0.75)
percentiles = df_sal['salario_kusd'].quantile([0.1,0.25,0.5,0.75,0.9])

q1, q2, q3, percentiles

## Tipos de distribuciones — ejemplos sintéticos
Generaremos datos sintéticos para ver claramente: normal, sesgada a la derecha, bimodal y uniforme.

In [ ]:
# Normal
x_norm = np.random.normal(50, 10, 1000)
# Sesgada a la derecha (log-normal)
x_right = np.random.lognormal(mean=3.7, sigma=0.4, size=1000)
# Bimodal: mezcla de dos normales
x_bimodal = np.concatenate([np.random.normal(30,5,600), np.random.normal(60,5,400)])
# Uniforme
x_unif = np.random.uniform(20,80,1000)

fig, axs = plt.subplots(2,2, figsize=(12,8))
axs = axs.ravel()
axs[0].hist(x_norm, bins=30)
axs[0].set_title('Normal')
axs[1].hist(x_right, bins=30)
axs[1].set_title('Sesgada a la derecha (log-normal)')
axs[2].hist(x_bimodal, bins=30)
axs[2].set_title('Bimodal')
axs[3].hist(x_unif, bins=30)
axs[3].set_title('Uniforme')

for ax in axs:
    ax.set_xlabel('Valor')
    ax.set_ylabel('Frecuencia')
plt.tight_layout()
plt.show()

## Correlaciones — ejemplos y visualización
Calcularemos la matriz de correlación entre variables numéricas y mostraremos una visualización con `imshow` y barras de color.

In [ ]:
# Correlación en df_sal
num_cols = df_sal.select_dtypes(include=[np.number])
cor_mat = num_cols.corr()

# Visualizar con imshow
plt.figure()
plt.imshow(cor_mat, vmin=-1, vmax=1)
plt.colorbar()
plt.xticks(range(len(cor_mat.columns)), cor_mat.columns, rotation=45)
plt.yticks(range(len(cor_mat.columns)), cor_mat.columns)
plt.title('Matriz de correlación (df_sal)')
plt.show()

cor_mat

### Correlación no implica causalidad — ejemplo con variable oculta
Crearemos un ejemplo simple donde dos variables están correlacionadas por una tercera variable latente (edad -> experiencia -> salario).

In [ ]:
# Ejemplo: variable latente
# Generamos A: edad, B: experiencia derivada de A, C: salario función de B
A = np.random.randint(20,60,200)  # edad
B = np.clip((A - 20) + np.random.normal(0,2,200),0,None)  # experiencia
C = 10 + 1.5 * B + np.random.normal(0,4,200)  # salario

# Crear dataframe y ver correlaciones
df_latent = pd.DataFrame({'edad':A, 'experiencia':B, 'salario':C})
print(df_latent.corr())

# Observación: edad y salario pueden correlacionar, pero la causalidad es experiencia -> salario


## Detección de outliers — métodos prácticos
Mostraremos z-score y método IQR, y cómo documentar decisiones (eliminar o investigar).

In [ ]:
# Z-score method
z_scores = np.abs(stats.zscore(df_sal['salario_kusd']))
outliers_z = df_sal[z_scores > 3]

# IQR method
Q1 = df_sal['salario_kusd'].quantile(0.25)
Q3 = df_sal['salario_kusd'].quantile(0.75)
IQR = Q3 - Q1
outliers_iqr = df_sal[(df_sal['salario_kusd'] < (Q1 - 1.5*IQR)) | (df_sal['salario_kusd'] > (Q3 + 1.5*IQR))]

len(outliers_z), len(outliers_iqr), outliers_iqr.head()

## Caso práctico — E-commerce: analizar ventas vs gasto en publicidad
Generaremos un dataset de 24 meses con `visitas`, `gasto_pub` (kUSD), `ventas` (kUSD), `conversion_rate`.
Ejemplos de análisis: mes con mayor eficiencia (ventas/gasto_pub), correlación visitas-ventas, ticket promedio.

In [ ]:
# Generar dataset e-commerce
np.random.seed(1)
months = pd.date_range('2023-01-01', periods=24, freq='M')
visitas = np.random.poisson(20000, 24) + np.linspace(0,5000,24).astype(int)
gasto_pub = np.round(np.random.normal(30,8,24),1)  # kUSD
conversion_rate = np.clip(0.01 + np.random.normal(0,0.002,24) + (gasto_pub-30)*0.0005, 0.005, 0.05)
ventas = np.round(visitas * conversion_rate * 0.05,2)  # simplificación (en miles USD unitless scale)

df_ecom = pd.DataFrame({'mes': months, 'visitas': visitas, 'gasto_pub_kusd': gasto_pub, 'conversion_rate': np.round(conversion_rate,4), 'ventas_kusd': ventas})
df_ecom['eficiencia'] = (df_ecom['ventas_kusd'] / df_ecom['gasto_pub_kusd']).replace([np.inf, -np.inf], np.nan)

df_ecom.head()

In [ ]:
# Mes con mayor eficiencia
best = df_ecom.loc[df_ecom['eficiencia'].idxmax()]
best

# Scatter visitas vs ventas
plt.figure()
plt.scatter(df_ecom['visitas'], df_ecom['ventas_kusd'])
plt.xlabel('Visitas')
plt.ylabel('Ventas (kUSD)')
plt.title('Visitas vs Ventas')
plt.show()

# Correlación
print('Correlación visitas-ventas:', df_ecom[['visitas','ventas_kusd']].corr().iloc[0,1])

# Ticket promedio ejemplo: ventas / (asumamos # órdenes)
ordenes = (df_ecom['visitas'] * df_ecom['conversion_rate']).round().astype(int)
df_ecom['ticket_promedio_kusd'] = (df_ecom['ventas_kusd'] / ordenes).replace([np.inf, -np.inf], np.nan)

plt.figure()
plt.plot(df_ecom['mes'], df_ecom['ticket_promedio_kusd'], marker='o')
plt.xticks(rotation=45)
plt.title('Ticket promedio (kUSD) por mes')
plt.ylabel('Ticket promedio (kUSD)')
plt.show()

df_ecom[['mes','visitas','gasto_pub_kusd','ventas_kusd','eficiencia','ticket_promedio_kusd']].head()

## Ejercicios para el aula
1. Calcular y comparar media/mediana por `nivel_educativo` en el dataset de salarios.
2. Identificar outliers por departamento y justificar si se eliminan o no.
3. En el caso e-commerce: encontrar el mes con mayor crecimiento de ventas vs mes anterior y proponer hipótesis.
4. Diseñar un pequeño experimento para comprobar causalidad entre gasto en publicidad y ventas (qué datos recolectarías).

In [ ]:
# Guardar datasets para uso en clase (opcional)
df_sal.to_csv('/mnt/data/salarios_ejemplo.csv', index=False)
df_ecom.to_csv('/mnt/data/ecommerce_ejemplo.csv', index=False)

print('Archivos guardados:')
print('/mnt/data/salarios_ejemplo.csv')
print('/mnt/data/ecommerce_ejemplo.csv')